In [1]:
from pathlib import Path
import pandas as pd
import os
import re
import logging

In [2]:
import ast
import io
import itertools
import zipfile
from collections import Counter
from logging import Logger
from pathlib import Path
from typing import List, Union

import numpy as np
import pandas as pd
import regex


def fill_na(cell, fill=[]):
    """
    Fill elements in pd.DataFrame with `fill`.
    """
    if hasattr(cell, "__iter__"):
        if isinstance(cell, str):
            if cell == "nan":
                return fill
            return cell
        nas = [True if el == "nan" or pd.isna(el) else False for el in cell]
        if all(nas):
            return fill
        return cell
    if pd.isna(cell):
        return fill
    return cell


def process_lote(el):
    """ 
    Convert string representation of a list into a list. Used to process the columns 'ProcurementProjectLot.ProcurementProject.Name' and 'ProcurementProjectLot.ID'.

    Parameters
    ----------
    el : str
        String representation of a list.

    Returns
    -------
    el : list
        List of elements.
    """
    if pd.notna(el):
        try:
            aux = ast.literal_eval(el[0])
            if isinstance(aux, float):
                aux = [int(aux)]
            return aux
        except (SyntaxError, ValueError):
            return [el[0]]
    else:
        return el


def melt_two_series(s1, s2):
    """ 
    Melt two series into a DataFrame, expanding rows to match elements in the series, 
    effectively transforming a single row into multiple rows. This function is useful 
    when dealing with columns containing lists, such as 'ProcurementProjectLot.ProcurementProject.Name' 
    and 'ProcurementProjectLot.ID'. 

    Parameters
    ---------- 
    s1 : pd.Series
        First series to melt.
    s2 : pd.Series)
        Second series to melt.

    Returns
    -------
    pd.DataFrame: A DataFrame with melted series, with each row containing metadata from the original DataFrame but for each element in the series.

    Example
    -------
    >>> s1 = pd.Series([['A', 'B'], ['C'], ['D', 'E', 'F']])
    >>> s2 = pd.Series([1, 2, 3])
    >>> melt_two_series(s1, s2)
      lot_name  lot_id
    0        A       1
    1        B       1
    2        C       2
    3        D       3
    4        E       3
    5        F       3
    """
    lengths_s1 = s1.str.len().values

    flat_s1 = [i for i in itertools.chain.from_iterable(s1.values.tolist())]
    flat_s2 = [i for i in itertools.chain.from_iterable(s2.values.tolist())]

    idx_s1 = np.repeat(s1.index.values, lengths_s1)

    return pd.DataFrame({'lot_name': flat_s1, 'lot_id': flat_s2}, index=idx_s1)


In [3]:
dir_data = Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data")
dir_text_metadata = Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/test")

path_parquets = Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/all_processed")
path_place_without_lote_processed_title = path_parquets / "trf_lote_es.parquet"

path_metadata = Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/metadata")
path_in = path_metadata / "insiders.parquet"
path_out = path_metadata / "outsiders.parquet"
path_min = path_metadata / "minors.parquet"

### READ PROCESSED DATA

In [4]:
processed = pd.read_parquet(path_place_without_lote_processed_title)
processed

,id_tm,raw_text,lemmas
identifier,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/499,0,Reforma de elementos de ventilación exterior d...,reforma elemento ventilación exterior aire_aco...
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/498,1,Servicios de calibrado y certificado de dos de...,calibrado certificado detector portátil gas tr...
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/496,3,Redacción EPIA - Legalización antena emisora d...,epia legalización antena emisora_radio narcea
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/495,4,Obras de reparación del Centro de Información ...,centro información naturaleza rellano
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/494,5,Diversos materiales para el acondicionamiento ...,rotonda arriat decoracion jardinera maceta via...
...,...,...,...
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/4,3110256,concesion de servicios de peluquerías en 10 cp...,concesion cpas dependiente dt
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/3,3110257,Servicios y suministros para la seguridad inte...,integral veiasa
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/2,3110258,Contrato de concesión de servicios para la ges...,concesión integral cfa


In [5]:
def merge_data(
    dir_data,
    dir_text_metadata,
    merge_dfs= ["minors", "insiders", "outsiders"],
    use_lot_info: bool = True,
    eliminate_duplicates: bool = True,
    logger = None,
):
    """
    Merge original data parquet files into single dataframe
    """
    dir_data = Path(dir_data)
    dfs = []
    origins = []

    if dir_data.suffix == ".zip":
        # If it's a zip file, create a zipfile.Path and find the specific folder within it.
        with zipfile.ZipFile(dir_data, "r") as zip_ref:
            all_files = zip_ref.namelist()
            for d in merge_dfs:
                file_path = f"metadata/{d}.parquet"
                if file_path in all_files:
                    pq_file = io.BytesIO(zip_ref.read(file_path))
                    dfs.append(pd.read_parquet(pq_file))
                    origins.append(d)
                elif logger:
                    logger.warning(
                        f"File {d}.parquet does not exist in the zip file, skipping."
                    )
                else:
                    continue
    else:
        # If it's a directory, create a Path and find the specific folder within it.
        for d in merge_dfs:
            file_path = dir_data.joinpath(f"metadata/{d}.parquet")
            logger.warning(f"File {file_path.as_posix()} does not exist, skipping.")
            if file_path.exists():
                dfs.append(pd.read_parquet(file_path))
                origins.append(d)
            elif logger:
                logger.warning(f"File {d}.parquet does not exist, skipping.")
            else:
                continue
    if not dfs:
        if logger:
            logger.error("No dataframes to merge.")
        return

    # Unify texts from all sources
    dfs_text = []
    for df, d in zip(dfs, origins):
        # Reset index and rename to common identifier: new index is generated as the concatenation of 'zip', 'file name', 'entry'
        index_names = df.index.names
        orig_cols = df.columns
        df.reset_index(inplace=True)
        df["identifier"] = df[index_names].astype(str).agg("/".join, axis=1)
        df.set_index("identifier", inplace=True)
        df = df[orig_cols]

        # Select text columns and rename them
        def join_str(x): return ".".join([el for el in x if el])
        joint_cnames = {join_str(c): c for c in df.columns}
        reverse_joint_cnames = {v: k for k, v in joint_cnames.items()}

        if not use_lot_info:
            cols_keep = ["title", "summary", "text", "origin"]
            cols_order = ["id_tm", "title", "summary", "text", "origin"]
            # If we don't want to use lot info, we only need to select the text columns 'summary' and 'title'
            text_cols = sorted(
                [
                    v for k, v in joint_cnames.items()
                    if "summary" in k or "title" in k
                ]
            )
            

            df_text = df.loc[:, text_cols]
            use_cols = [
                reverse_joint_cnames[c].split(".", 1)[-1]
                for c in text_cols
            ]
            df_text.columns = use_cols

            # Define columns that will be used as text
            columns_for_text = ["title", "summary"]
        else:
            print("USING LOT INFO")
            cols_keep = ["title", "summary", "lot_name", "text", "origin"]
            cols_order = ["id_tm", "title", "summary", "lot_name", "text", "origin"]
            
            # If we want to use lot info, we need to select the text columns 'summary' and 'title' and the columns 'ProcurementProjectLot.ProcurementProject.Name' and 'ProcurementProjectLot.ID'
            text_cols = sorted(
                [
                    v for k, v in joint_cnames.items()
                    if "summary" in k
                    or "title" in k
                    or "id" in k
                    or "ContractFolderStatus.ProcurementProjectLot.ProcurementProject.Name" in k
                    or "ContractFolderStatus.ProcurementProjectLot.ID" in k
                ]
            )

            df_text = df.loc[:, text_cols]
            use_cols = [
                reverse_joint_cnames[c].split(".", 1)[-1]
                for c in text_cols
            ]
            df_text.columns = use_cols

            if "ProcurementProjectLot.ID" in use_cols:

                # Rename columns for better readability
                rename_columns = {
                    "ProcurementProjectLot.ID": "lot_id",
                    "ProcurementProjectLot.ProcurementProject.Name": "lot_name",
                }
                df_text = df_text.rename(columns=rename_columns)

                # Convert columns 'lot_name' and 'lot_id' to lists
                df_text["lot_name"] = df_text["lot_name"].apply(process_lote)
                df_text["lot_id"] = df_text["lot_id"].apply(process_lote)

                # Melt the columns 'lot_name' and 'lot_id' into a single DataFrame
                df_text = melt_two_series(df_text['lot_name'], df_text['lot_id']).join(
                    df_text.drop(['lot_name', 'lot_id'], axis=1))

                # Set index to 'identifier' lost during melt
                df_text.index.names = ['identifier']

                # Redefine identifier to include lot_id if it exists
                orig_cols = ["title", "summary", "lot_name"]
                df_text.reset_index(inplace=True)
                df_text["identifier"] = df_text.apply(lambda row: '/'.join([str(row['identifier']), str(
                    row['lot_id'])]) if row['lot_id'] != "nan" else row['identifier'], axis=1)
                df_text.set_index("identifier", inplace=True)

                # Define columns that will be used as text
                columns_for_text = ["title", "summary", "lot_name"]#"id"
            else:
                rename_columns = {
                    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv",
                }
                df_text = df_text.rename(columns=rename_columns)
                
                df_text["lot_name"] = len(df_text) * np.nan

        df_text["text"] = (
            df_text[columns_for_text]
            .applymap(fill_na, fill=None)
            .agg(lambda x: ". ".join([str(el) for el in x if el]), axis=1)
        )

        if eliminate_duplicates:
            df_text = df_text.drop_duplicates(subset=['text'])
        
        df_text["origin"] = [d] * len(df_text)

        dfs_text.append(df_text)

    # Concatenate and save as unique DataFrame-
    df_text = pd.concat(dfs_text)[cols_keep]
    if "lot_name" in cols_keep:
        mask = df_text['lot_name'].apply(lambda x: isinstance(x, int))
        df_text.loc[mask, 'lot_name'] = df_text.loc[mask, 'lot_name'].astype(str)
    dir_text_metadata.parent.mkdir(parents=True, exist_ok=True)
    df_text["id_tm"] = np.arange(len(df_text))
    df_text = df_text[cols_order]
    df_text.to_parquet(dir_text_metadata, engine="pyarrow")
    return df_text

In [6]:
logger = logging.Logger("test")
df = merge_data(dir_data, dir_text_metadata, use_lot_info = False, logger = logger)

File /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/metadata/minors.parquet does not exist, skipping.
File /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/metadata/insiders.parquet does not exist, skipping.
File /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/metadata/outsiders.parquet does not exist, skipping.


In [7]:
df

,id_tm,title,summary,text,origin
identifier,,,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/499,0,Reforma de elementos de ventilación exterior d...,Id licitación: 000103/2017-1069; Órgano de Con...,Reforma de elementos de ventilación exterior d...,minors
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/498,1,Servicios de calibrado y certificado de dos de...,Id licitación: 29-2017-II; Órgano de Contratac...,Servicios de calibrado y certificado de dos de...,minors
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/497,2,"Patrocinio menor proyecto "" Activitats C.I.N.E""",Id licitación: 013-07-2018; Órgano de Contrata...,"Patrocinio menor proyecto "" Activitats C.I.N.E...",minors
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/496,3,Redacción EPIA - Legalización antena emisora d...,Id licitación: CON/2017/51; Órgano de Contrata...,Redacción EPIA - Legalización antena emisora d...,minors
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/495,4,Obras de reparación del Centro de Información ...,Id licitación: 000047/2017-1069; Órgano de Con...,Obras de reparación del Centro de Información ...,minors
...,...,...,...,...,...
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/4,3110256,concesion de servicios de peluquerías en 10 cp...,Id licitación: CONTR 2023 0000922771; Órgano d...,concesion de servicios de peluquerías en 10 cp...,outsiders
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/3,3110257,Servicios y suministros para la seguridad inte...,Id licitación: CR050-23-087B; Órgano de contra...,Servicios y suministros para la seguridad inte...,outsiders
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/2,3110258,Contrato de concesión de servicios para la ges...,Id licitación: CONTR 2023 0001215494; Órgano d...,Contrato de concesión de servicios para la ges...,outsiders


In [8]:
processed_duplicates = processed[processed.duplicated(subset="raw_text")]

In [9]:
processed_duplicates

,id_tm,raw_text,lemmas
identifier,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/133,366,Mecanizado de piezas,mecanizado_pieza
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/21,478,Traslado y depósito de vehículos.,traslado depósito
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/16,483,Traslado y depósito de vehículos.,traslado depósito
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/9,490,Traslado y depósito de vehículos.,traslado depósito
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_11.atom/496,503,Traslado y depósito de vehículos.,traslado depósito
...,...,...,...
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/78,3110182,Contrato privado de seguros de riesgos permane...,privado seguro riesgo permanente ayuntamiento
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/72,3110188,Suministro de licencias y los servicios de man...,aplicación_informático jira_software
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores.atom/12,3110248,Suministro de modificador de fricción en el co...,modificador_fricción contacto metro


In [14]:
processed_duplicates[0:50]

,id_tm,raw_text,lemmas
identifier,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/133,366,Mecanizado de piezas,mecanizado_pieza
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/21,478,Traslado y depósito de vehículos.,traslado depósito
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/16,483,Traslado y depósito de vehículos.,traslado depósito
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/9,490,Traslado y depósito de vehículos.,traslado depósito
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_11.atom/496,503,Traslado y depósito de vehículos.,traslado depósito
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_11.atom/490,509,Traslado y depósito de vehículos.,traslado depósito
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_11.atom/333,666,Material de papelería para establecimiento de ...,papelería establecimiento_mercafresh mercasa
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_11.atom/332,667,Material de papelería para establecimiento de ...,papelería establecimiento_mercafresh mercasa
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_11.atom/331,668,Material de papelería para establecimiento de ...,papelería establecimiento_mercafresh mercasa


In [10]:
checking = processed_duplicates.merge(df, how="left", on="id_tm")

In [11]:
checking[0:50]

,id_tm,raw_text,lemmas,title,summary,text,origin
0,366,Mecanizado de piezas,mecanizado_pieza,Mecanizado de piezas,Id licitación: 324/17; Órgano de Contratación:...,Mecanizado de piezas. Id licitación: 324/17; Ó...,minors
1,478,Traslado y depósito de vehículos.,traslado depósito,Traslado y depósito de vehículos.,Id licitación: LE-2017/49 CM SE; Órgano de Con...,Traslado y depósito de vehículos.. Id licitaci...,minors
2,483,Traslado y depósito de vehículos.,traslado depósito,Traslado y depósito de vehículos.,Id licitación: LE-2017/50 CM SE; Órgano de Con...,Traslado y depósito de vehículos.. Id licitaci...,minors
3,490,Traslado y depósito de vehículos.,traslado depósito,Traslado y depósito de vehículos.,Id licitación: LE-2017/51 CM SE; Órgano de Con...,Traslado y depósito de vehículos.. Id licitaci...,minors
4,503,Traslado y depósito de vehículos.,traslado depósito,Traslado y depósito de vehículos.,Id licitación: LE-2017/53 CM SE; Órgano de Con...,Traslado y depósito de vehículos.. Id licitaci...,minors
5,509,Traslado y depósito de vehículos.,traslado depósito,Traslado y depósito de vehículos.,Id licitación: LE-2017/55 CM SE; Órgano de Con...,Traslado y depósito de vehículos.. Id licitaci...,minors
6,666,Material de papelería para establecimiento de ...,papelería establecimiento_mercafresh mercasa,Material de papelería para establecimiento de ...,Id licitación: 01/16 DE-2/7; Órgano de Contrat...,Material de papelería para establecimiento de ...,minors
7,667,Material de papelería para establecimiento de ...,papelería establecimiento_mercafresh mercasa,Material de papelería para establecimiento de ...,Id licitación: 01/16 DE-3/7; Órgano de Contrat...,Material de papelería para establecimiento de ...,minors
8,668,Material de papelería para establecimiento de ...,papelería establecimiento_mercafresh mercasa,Material de papelería para establecimiento de ...,Id licitación: 01/16 DE-4/7; Órgano de Contrat...,Material de papelería para establecimiento de ...,minors
9,669,Material de papelería para establecimiento de ...,papelería establecimiento_mercafresh mercasa,Material de papelería para establecimiento de ...,Id licitación: 01/16 DE-5/7; Órgano de Contrat...,Material de papelería para establecimiento de ...,minors


#### Ahora con lot info

In [12]:
df = merge_data(dir_data, dir_text_metadata, use_lot_info = True, logger = logger)

File /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/metadata/minors.parquet does not exist, skipping.
File /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/metadata/insiders.parquet does not exist, skipping.
File /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/metadata/outsiders.parquet does not exist, skipping.


USING LOT INFO
USING LOT INFO
USING LOT INFO


In [13]:
df

,id_tm,title,summary,lot_name,text,origin
identifier,,,,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes.atom/0,0,Actuacion de desmontado de cubierta de placas ...,Id licitación: 2.18/32619.5069/01; Órgano de C...,nan,Actuacion de desmontado de cubierta de placas ...,minors
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes.atom/10,1,Servicio de mantenimiento de una puerta girato...,Id licitación: 37/CM-02/19; Órgano de Contrata...,nan,Servicio de mantenimiento de una puerta girato...,minors
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes.atom/101,2,Servivios tecnologia GESTIONA: suscripcion man...,Id licitación: FTRA-0254/2018; Órgano de Contr...,nan,Servivios tecnologia GESTIONA: suscripcion man...,minors
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes.atom/107,3,"Suministro de equipos de sobremesa, portátiles...",Id licitación: 18.115.GS999.AI.01; Órgano de C...,nan,"Suministro de equipos de sobremesa, portátiles...",minors
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes.atom/108,4,Realización de backup remoto en las oficinas d...,Id licitación: 18.114.GS999.AI.01; Órgano de C...,nan,Realización de backup remoto en las oficinas d...,minors
...,...,...,...,...,...,...
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores_20240201_040018_1.atom/95,3360668,servicio de punto de encuentro familiar en la ...,Id licitación: CONTR 2023 0000927905; Órgano d...,nan,servicio de punto de encuentro familiar en la ...,outsiders
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores_20240201_040018_1.atom/96/1,3360669,0001181/2023 contr 2023 1204117 servicio de ma...,Id licitación: +6.65MUHQ7; Órgano de contratac...,Mantenimiento y reparación de sistemas para pr...,0001181/2023 contr 2023 1204117 servicio de ma...,outsiders
PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores_20240201_040018_1.atom/97,3360670,renting vehiculo 5 plazas para el delegado t. ...,Id licitación: CONTR 2023 0001053296; Órgano d...,nan,renting vehiculo 5 plazas para el delegado t. ...,outsiders
